In [1]:
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.remote.webelement import WebElement
from selenium import webdriver
from selenium.webdriver.remote import webelement
from selenium.common.exceptions import StaleElementReferenceException, ElementClickInterceptedException
from IPython.display import Image, display
import os
import sys
from time import sleep

import pandas as pd
import gspread
from google.auth import default
from google.auth.exceptions import RefreshError

In [74]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-infobars")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-browser-side-navigation")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=options)
driver.get("https://www.google.com")

In [72]:
def scrape_job_data(driver,link):
    driver.get(link)
    # Tìm tất cả các job card trên trang
    job_cards = driver.find_elements(By.XPATH, '//div[contains(@class, "JobCard_jobCardContainer__arQlW")]')

    # Kiểm tra số lượng job card tìm được
    print(f"Đã tìm thấy {len(job_cards)} công việc.")
    all_jobs_data = []

    # Duyệt qua từng job card và lấy thông tin
    for index, job in enumerate(job_cards, start=1):
        try:
            company_name = job.find_element(By.XPATH, './/span[@class="EmployerProfile_compactEmployerName__9MGcV"]').text
        except NoSuchElementException:
            company_name = 'N/A'

        try:
            location = job.find_element(By.XPATH, './/div[@class="JobCard_location__Ds1fM"]').text
        except NoSuchElementException:
            location = 'N/A'

        try:
            job_title = job.find_element(By.XPATH, './/a[@class="JobCard_jobTitle__GLyJ1"]').text
        except NoSuchElementException:
            job_title = 'N/A'

        try:
            salary = job.find_element(By.XPATH, './/div[@class="JobCard_salaryEstimate__QpbTW"]').text
        except NoSuchElementException:
            salary = 'N/A'

        try:
            link_job = job.find_element(By.XPATH, './/a[@data-test="job-link"]').get_attribute('href')
        except NoSuchElementException:
            link_job = 'N/A'

        print(f"Công việc {index}:")
        print(f"  Tên công ty: {company_name}")
        print(f"  Vị trí: {location}")
        print(f"  Tiêu đề công việc: {job_title}")
        print(f"  Mức lương: {salary}")
        print(f"  Liên kết: {link_job}")
        print("-" * 50)
        
        # Lưu thông tin công việc
        all_jobs_data.append([link_job, company_name, job_title, salary, location])

    return all_jobs_data

In [68]:
import gspread
from google.oauth2.service_account import Credentials
try:
    # Đường dẫn đến tệp JSON của bạn
    SERVICE_ACCOUNT_FILE = 'snappy-charge-444008-u3-2d010cb44a06.json'

    # Phạm vi quyền truy cập
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    # Xác thực với Google Sheets API
    credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(credentials)

    # Mở Google Sheet bằng URL
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1OQgLQlWVfkZEy7K1_ywSFHA2cUNZyoZ2ldMF1RaNnZE/edit?usp=sharing'
    sheet = gc.open_by_url(spreadsheet_url)

    # Lấy trang tính đầu tiên
    work_sheet = sheet.get_worksheet(0)

    # Lấy hàng tiêu đề
    header = work_sheet.row_values(1)

    # Kiểm tra sự tồn tại của các tiêu đề cần thiết
    required_columns = ["Company name", "Title job", "Location job", "Salary"]
    for column in required_columns:
        if column not in header:
            print(f"Column '{column}' not found in the header.")
            exit(1)

    # Tìm vị trí các cột theo tiêu đề
    company_col = header.index("Company name") + 1  # Cột "Company name"
    job_col = header.index("Title job") + 1         # Cột "Title job"
    locate_col = header.index("Location job") + 1  # Cột "Location job"
    salary_col = header.index("Salary") + 1        # Cột "Salary"

    # Kiểm tra kết nối bằng cách in tiêu đề
    print(f"Successfully accessed sheet: {sheet.title}")
    print("Headers:", header)

    # Đọc tất cả dữ liệu
    data = work_sheet.get_all_records()
    print("Dữ liệu từ Google Sheet:", data)

except Exception as e:
    print(f"Error: {str(e)}")


Successfully accessed sheet: Crawling info job glassdoor
Headers: ['Link job', 'Company name', 'Title job', 'Salary', 'Location job']
Dữ liệu từ Google Sheet: []


In [69]:
import gspread
from google.oauth2.service_account import Credentials

def update_google_sheet(sheet, data):
    """
    Cập nhật Google Sheet với dữ liệu công việc đã thu thập.

    Args:
        sheet: Đối tượng Google Sheet.
        data: Danh sách các dữ liệu công việc để thêm.
    """
    try:
        # Kiểm tra xem dữ liệu có rỗng không
        if not data:
            print("Không có dữ liệu để cập nhật.")
            return

        # Chuẩn bị dữ liệu dưới dạng danh sách các danh sách
        # Mỗi danh sách con tương ứng với một hàng dữ liệu
        # Giả sử data là danh sách các danh sách hoặc danh sách các từ điển
        # Nếu data là danh sách các từ điển, chuyển đổi nó thành danh sách các danh sách
        if isinstance(data[0], dict):
            # Xác định thứ tự cột theo tiêu đề đã lấy trước đó
            headers = sheet.row_values(1)
            prepared_data = []
            for row in data:
                prepared_row = [
                    row.get("Link job", ""),
                    row.get("Company name", ""),
                    row.get("Title job", ""),
                    row.get("Salary", ""),
                    row.get("Location job", "")
                ]
                prepared_data.append(prepared_row)
        else:
            # Nếu data đã là danh sách các danh sách
            prepared_data = data

        # Sử dụng append_rows để thêm nhiều hàng cùng một lúc
        sheet.append_rows(prepared_data, value_input_option='RAW')
        print(f"Đã thêm thành công {len(prepared_data)} hàng vào Google Sheet.")

    except Exception as e:
        print(f"Lỗi khi cập nhật Google Sheet: {str(e)}")


In [75]:
link='https://www.glassdoor.com/Job/california-us-data-analyst-jobs-SRCH_IL.0,13_IS2280_KO14,26.htm'
jobs= scrape_job_data(driver, link)

Đã tìm thấy 30 công việc.
Công việc 1:
  Tên công ty: ThirdLove
  Vị trí: San Francisco, CA
  Tiêu đề công việc: Data Analyst
  Mức lương: $90K - $140K (Employer est.)
  Liên kết: https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=1136043&s=58&guid=00000193f19712148aad1ee93b8901d1&src=GD_JOB_AD&t=SR&vt=w&cs=1_5380ff59&cb=1734925030305&jobListingId=1009564330231&jrtk=5-yul1-0-1ifope4jv21g1003-82494dfce69da7bd
--------------------------------------------------
Công việc 2:
  Tên công ty: HockeyStack
  Vị trí: San Francisco, CA
  Tiêu đề công việc: Data Analyst
  Mức lương: $85K - $119K (Glassdoor est.)
  Liên kết: https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=1136043&s=58&guid=00000193f19712148aad1ee93b8901d1&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_c89c7ff4&cb=1734925030305&jobListingId=1009573838577&jrtk=5-yul1-0-1ifope4jv21g1003-914d5c666d1f9f4a
--------------------------------------------------
Công việc 3:
  Tên công ty: Exchange Bank
  Vị trí: Santa Rosa, CA
  Tiêu 

In [77]:
update_google_sheet(work_sheet, jobs)

Đã thêm thành công 30 hàng vào Google Sheet.
